In [16]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd

In [17]:
# 將人口和面積的原始資料讀入
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

sql_location = "SELECT * FROM raw_location"
sql_population = "SELECT * FROM raw_population"

df_loc = pd.read_sql(sql_location, engine)
df_popu = pd.read_sql(sql_population, engine)

print("資料讀取完成")

資料讀取完成


In [18]:
# 將兩個dataframe按照"市"及"區"欄位結合，並把無人口的地區刪除
df_main = df_loc.merge(df_popu, how='left', on=["city", "district"])

df_main.dropna(inplace=True)

folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\loc_population"
file = "loc_with_population_all.csv"
file_path = os.path.join(folder, file)

df_main.to_csv(file_path, index=False)

In [19]:
# 取出只有六都的資料
city_dict = {
    "新北市":"NTP",
    "臺北市":"TPE",
    "桃園市":"TYN",
    "臺中市":"TCH",
    "臺南市":"TNA",
    "高雄市":"KSH"
}

city_list = list(city_dict.keys())

df_6city = (df_main[df_main["city"].isin(city_list)]).copy()

In [20]:
# 增加id欄位並編號
df_6city["code"] = df_6city["city"].map(city_dict)

df_6city["seq"] = df_6city.groupby("code").cumcount() + 1

df_6city["seq"] = df_6city["seq"].apply(lambda x: f"{x:03d}")

df_6city["loc_id"] = df_6city["code"] + df_6city["seq"]

df_6city.drop(columns=["code", "seq"], axis=1, inplace=True)

In [21]:
# 重新調整欄位順序
new_col = ['loc_id', 'city', 'district', 'area(km^2)', 'household_count', 'population', 'male', 'female']

df_6city = df_6city[new_col]

rename_col = ['loc_id', 'city', 'district', 'area', 'household_count', 'population', 'male', 'female']

df_6city.columns = rename_col

In [22]:
df_6city.iloc[90:120]

,loc_id,city,district,area,household_count,population,male,female
93,TNA007,臺南市,東山區,124.92,8222.0,18558.0,9907.0,8651.0
94,TNA008,臺南市,麻豆區,53.97,17835.0,42962.0,21731.0,21231.0
95,TNA009,臺南市,下營區,33.53,9524.0,21812.0,11225.0,10587.0
96,TNA010,臺南市,六甲區,67.55,8617.0,20930.0,10769.0,10161.0
97,TNA011,臺南市,官田區,70.80,8706.0,20563.0,10497.0,10066.0
98,TNA012,臺南市,大內區,70.31,3666.0,8460.0,4531.0,3929.0
99,TNA013,臺南市,佳里區,38.94,23493.0,58348.0,28714.0,29634.0
100,TNA014,臺南市,學甲區,53.99,10272.0,24413.0,12511.0,11902.0
101,TNA015,臺南市,西港區,33.77,9637.0,24847.0,12529.0,12318.0
102,TNA016,臺南市,七股區,110.15,8136.0,20580.0,10645.0,9935.0


In [23]:
# 存檔地端
file_6city = "loc_with_population_6city.csv"
path_6city = os.path.join(folder, file_6city)

df_6city.to_csv(path_6city, index=False)

In [24]:
# 輸入資料庫
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

df_6city.to_sql(name="location", con=engine, index=False, if_exists="replace")

159